This notebook takes a list of route ids and calls the Mountain Project api to acquire the route information.

In [9]:
from pymongo import MongoClient
import json, re, requests, pprint, urllib, time
from bs4 import BeautifulSoup
from mp_api import MP
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
key_path = '../data/mp_api_key.txt'
with open(key_path, 'r') as key_text:
    key = key_text.read()

In [33]:
routes_path = '../data/route_ids.txt'
with open(routes_path, 'r') as routes_text:
    route_ids = routes_text.read().splitlines()
print(len(route_ids))

4982


In [34]:
client = MongoClient('localhost', 27017)
db = 'mountain_project'
collection = 'routes'
if collection in client[db].list_collection_names():
    (client[db])[collection].drop()

regional_routes = (client[db])['regional_routes']

for region in regional_routes.find():
    routes = region['data']
    for route in routes:
        (client[db])[collection].insert_one(route)

In [35]:
(client[db])[collection].count_documents({})

4982

In [36]:
new_routes_path = '../data/new_route_ids.txt'
with open(new_routes_path, 'r') as new_routes_text:
    route_ids = new_routes_text.read().splitlines()
print(len(route_ids))

75614


In [ ]:
client = MongoClient('localhost', 27017)
db = 'mountain_project'
collection = 'routes'

route_db = MP(key, client, db, collection)

for idx in range(100, len(route_ids), 100):
    print(idx)
    route_db.api_get_routes(route_ids[idx - 100: idx])
    last = idx
route_db.api_get_routes(route_ids[last:])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000


In [32]:
route_db.collection.count_documents({})

5082

In [ ]:
pprint.pprint(route_db.collection.find_one())